# Representing Text

We can represent text in many ways: character strings are a standard representation, but we can also create numerical representations of text. In this notebook we will discuss embeddings.

## Features

Features to any (machine learning) model can be continuous or categorical.

- We use continuous features to represent numerical values: income, number of times the user clicked on a link, prices, etc.
- Categorical features represent an instance of a class or category. They have a finite number of possible values: job title, genre of a movie, breed of a dog, etc.

## Embeddings

An **embedding** is a trained numerical representation of a categorical feature:

- We use the word *trained* to highlight that embeddings are learned during model training.
- Different models and training procedures can be used to obtained embeddings. Word2Vec and BERT embeddings, for example, are different and capture different characteristics of the features.

[OpenAI's documentation](https://platform.openai.com/docs/guides/embeddings) include a few uses of embeddings:


- Search: results are ranked by relevance to a query string.
- Clustering:  text strings are grouped by similarity.
- Recommendations:  items with related text strings are recommended.
- Anomaly detection:  outliers with little relatedness are identified.
- Diversity measurement: similarity distributions are analyze.
- Classification: text strings are classified by their most similar label.

## BERT Embeddings

Bert embeddings are computed based on three ingredients discussed below:

+ Token embeddings (also called word embeddings)
+ Positional embeddings
+ Token type embeddings (also called sentence embeddings)

![](img/02_bert_architecture.png)

### Tokenization

Embedding computations start with tokenization: representing the original text as tokens in a vocabulary. 

To illustrate the process, we can use the [`transformers`](https://huggingface.co/docs/transformers/en/index) library from [HuggingFace](https://huggingface.co/).

In [ ]:
import transformers

documents = ["cats are fun"]

tokenizer = transformers.BertTokenizer.from_pretrained(
    'bert-base-uncased')
tokens = tokenizer(documents)
print(tokens)


In the code snippet above, we used the `transformers` library to obtain the tokens that represent the phrase 'cats are fun'. The tokenizer returns a dictionary with an entry called `'input_ids'`, which contains an array of four integers. These integers are the positions of each token in the model's vocabulary. The vocabulary is [`'bert-base-uncased'`](https://huggingface.co/bert-base-uncased/blob/main/vocab.txt) and it is applied using the method `.from_pretrained()`.

We can show the vocabulary entries with:

In [ ]:
print(f"Index of 'cats': {tokenizer.vocab['cats']}")
print(f"Index of 'are': {tokenizer.vocab['are']}")
print(f"Index of 'fun': {tokenizer.vocab['fun']}")

IDs 101 and 102 are special tokens:

- ID 101 is the `[CLS]` token, indicating the begginning of a sequence.
- ID 102 is the `[SEP]` token, indicating the end of a sequence.

They are inserted automatically to the output of the BERT tokenizer. The BERT Tokenizer includes 30,522 unique tokens. In addition, the BERT tokenizer handles unkown tokens, `[UNK]`, using techniques such as WordPiece. You can read more about this tokenizer in [BERT Tokenization (Nowak, 2023)](https://tinkerd.net/blog/machine-learning/bert-tokenization/) and Mastering Text Similarity ([Guadagnolo, 2024](https://medium.com/eni-digitalks/mastering-text-similarity-combining-embedding-techniques-and-distance-metrics-98d3bb80b1b6)).



### Token Embeddings

The tokens obtained from the previous step are mapped to the model's precomputed embeddings. For each token in the model vocabulary, there is an embedding vector.

<div>
<img src="img/02_skip_gram_architecture.png" width="700"/>
</div>

Image source: Mastering Text Similarity ([Guadagnolo, 2024](https://medium.com/eni-digitalks/mastering-text-similarity-combining-embedding-techniques-and-distance-metrics-98d3bb80b1b6))

In [ ]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
embedding_layer = model.embeddings
embedding_layer.word_embeddings.weight

In [ ]:
embedding_layer.word_embeddings.weight.shape

The attribute `.weight` of the embedding layer shows the actual embeddings. It is a matrix of 30,522 rows and 768 columns (the object, in reality, is a 2-dimenional vector). 

+ The number of rows is equal to the size of the model vocabulary.
+ The number of columns is the hidden size or the size of the model's internal representation. 

In [ ]:
tokens = tokenizer(['cats are fun'])
input_ids = tokens.input_ids[0]
input_ids

In [ ]:
doc_embeddings = embedding_layer.word_embeddings.weight[input_ids]
doc_embeddings

### Position Embeddings

In addition to token embeddings, the BERT model also keeps track of positions through position embeddings. In contrast with token embeddings, position embeddings have shape (512, 768). This is because the BERT model can only take up to 512 tokens at a time.

In [ ]:
embedding_layer.position_embeddings.weight

In [ ]:
embedding_layer.position_embeddings.weight.shape

### Token Type Embeddings

Token or Segment Type embeddings. BERT was trained to solve Next Sentence Prediciton. Given two sentences, A and B, BERT was trained to determine if B logically follows A.

In [ ]:
embedding_layer.token_type_embeddings

### The Embedding Layer

The embedding layer takes a list of token ids and converts them to embeddings that combine the three types discussed above: token, position, and type embeddings.

In [ ]:
import torch
tokens = tokenizer(['cats are fun'])
final_embeddings = embedding_layer(input_ids = torch.tensor(tokens.input_ids))
final_embeddings